In [1]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

In [2]:
import pandas as pd
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer


/home/heejet/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = QdrantClient(url="http://localhost:6333")
collection_name = "no_coordinates_collection"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    

/home/heejet/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/heejet/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
# I USED THIS FOR DEBUGGING
embeddings = model.encode('900 Embarcadero Del Mar')
print(len(embeddings))
output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value='Hotels',
                    ),
                )
            ]
        ),
        query_vector=embeddings,
        limit=3,
    )
print(output)

384
[ScoredPoint(id='5e744e0c-cee7-5cf6-b25f-bf4bbf6a6dd2', version=8, score=0.5553415, payload={'categories': ['Hotels', 'Hotels & Travel', 'Event Planning & Services'], 'full_address': '424 Por La Mar Santa Barbara 93103', 'hours': None, 'name': 'Parkside Inn', 'review_count': 42, 'stars': 3.0}, vector=None, shard_key=None), ScoredPoint(id='c557b509-606f-5a9e-9435-1bdc0d65cc07', version=45, score=0.53008175, payload={'categories': ['Venues & Event Spaces', 'Hotels & Travel', 'Event Planning & Services', 'Hotels', 'Wedding Planning'], 'full_address': '900 San Ysidro Ln Santa Barbara 93108', 'hours': {'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', 'Thursday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'Wednesday': '0:0-0:0'}, 'name': 'San Ysidro Ranch', 'review_count': 129, 'stars': 4.5}, vector=None, shard_key=None), ScoredPoint(id='5faa0e46-12ef-5ef6-92d2-43a12cee5e48', version=17, score=0.5225229, payload={'categories': ['Hotels', 'Hotels & Travel', 'Eve

# Define your tools!

For this we create a Langchain tool for retrieving weather information and use the `convert_to_openai_function` to create the function definition that we will feed into the model

In [38]:
class SearchInput(BaseModel):
    location: str = Field(description="Current location of user.")
    Category: str = Field(description="Type of business that the user wants to go.")
@tool("get_category", args_schema=SearchInput)
def get_category(location: str, Category: str) -> str:
    """Get a business recommendation given location and category of user's choice.\
        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea """
    print(location, Category)
    address_embedding = model.encode(location)
    # print(address_embedding)
    output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value=Category,
                    ),
                )
            ]
        ),
        
        query_vector=address_embedding,
        limit=3,
    )

    print('Query output:',output)
    best_point = max(output, key=lambda x: x.score)
    restaurant_name = best_point.payload['name']
    restaurant_address = best_point.payload['full_address']

    print(f"The best-rated restaurant is {restaurant_name} located at {restaurant_address}.")

    return {"Name of restaurant": restaurant_name ,"Location": restaurant_address}

tools = [get_category]
functions = [convert_to_openai_function(t) for t in tools]
functions

[{'name': 'get_category',
  'description': 'get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'Current location of user.',
     'type': 'string'},
    'Category': {'description': 'Type of business that the user wants to go.',
     'type': 'string'}},
   'required': ['location', 'Category']}}]

# Invoke the Model

Now we can prompt the model and pass the functions 🥳

In [39]:
import ollama
import json

SYSTEM_PROMPT = f"""
You are an navigation assistant with access to these functions -
{json.dumps(functions, indent=4)}
"""

print (SYSTEM_PROMPT)


You are an navigation assistant with access to these functions -
[
    {
        "name": "get_category",
        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user's choice.        e.g. \"Name of restaurant\": Helena Avenue Bakery, \"Location\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\"Category\": Food, Restaurants, Salad, Coffee & Tea",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "Current location of user.",
                    "type": "string"
                },
                "Category": {
                    "description": "Type of business that the user wants to go.",
                    "type": "string"
                }
            },
            "required": [
                "location",
                "Category"
            ]
        }
    }
]



In [40]:
import json
import re
from typing import Optional, Dict

def parse_function_call(input_str: str) -> Optional[Dict[str, any]]:
    """
    Parses a text string to find and extract a function call.
    The function call is expected to be in the format:
    <functioncall> {"name": "<function_name>", "arguments": "<arguments_json_string>"}

    Args:
        input_str (str): The text containing the function call.

    Returns:
        Optional[Dict[str, any]]: A dictionary with 'name' and 'arguments' if a function call is found,
                                  otherwise None.
    """
    # Regex pattern to extract 'name' and 'arguments'
    pattern = r'"name":\s*"([^"]+)",\s*"arguments":\s*\'(.*?)\''

    # Search with regex
    match = re.search(pattern, input_str)
    print(match)
    if match:
        try:
            name = match.group(1)
            arguments_str = match.group(2)
            arguments = json.loads(arguments_str)
            return {"name": name, "arguments": arguments}
        except json.JSONDecodeError:
            return None
    return None


In [45]:
messages = [
     {'role': 'system','content': SYSTEM_PROMPT}, 
     {'role': 'user','content': 'Im at 900 Embarcadero Del Mar, any hotels nearby?'},
]

response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
message = (response['message']['content'])

In [48]:
message

'<functioncall> {"name": "get_category", "arguments": \'{"location": "900 Embarcadero Del Mar", "Category": "Hotels"}\'}'

# Call the Function

Here we pull the function out of `kwargs` and call our tool with the arguments

In [49]:
messages.append({'role': 'assistant', 'content': message}) # add ai response to history

function_call = parse_function_call(message) # parse out function call name and args into json
print(function_call)
if function_call and function_call.get("name") == "get_category":
    args = function_call.get("arguments")
    restaurant = get_category.run(args)

restaurant

<re.Match object; span=(16, 116), match='"name": "get_category", "arguments": \'{"location>
{'name': 'get_category', 'arguments': {'location': '900 Embarcadero Del Mar', 'Category': 'Hotels'}}
900 Embarcadero Del Mar Hotels
Query output: [ScoredPoint(id='5e744e0c-cee7-5cf6-b25f-bf4bbf6a6dd2', version=8, score=0.5553415, payload={'categories': ['Hotels', 'Hotels & Travel', 'Event Planning & Services'], 'full_address': '424 Por La Mar Santa Barbara 93103', 'hours': None, 'name': 'Parkside Inn', 'review_count': 42, 'stars': 3.0}, vector=None, shard_key=None), ScoredPoint(id='c557b509-606f-5a9e-9435-1bdc0d65cc07', version=45, score=0.53008175, payload={'categories': ['Venues & Event Spaces', 'Hotels & Travel', 'Event Planning & Services', 'Hotels', 'Wedding Planning'], 'full_address': '900 San Ysidro Ln Santa Barbara 93108', 'hours': {'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', 'Thursday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'Wednesday': '0:0-0:0'}, '

{'Name of restaurant': 'Parkside Inn',
 'Location': '424 Por La Mar Santa Barbara 93103'}

In [50]:
messages.append({'role': 'user', 'content': 'Function Response: ' + str(restaurant)})
messages

[{'role': 'system',
  'content': '\nYou are an navigation assistant with access to these functions -\n[\n    {\n        "name": "get_category",\n        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. \\"Name of restaurant\\": Helena Avenue Bakery, \\"Location\\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\\"Category\\": Food, Restaurants, Salad, Coffee & Tea",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "location": {\n                    "description": "Current location of user.",\n                    "type": "string"\n                },\n                "Category": {\n                    "description": "Type of business that the user wants to go.",\n                    "type": "string"\n                }\n            },\n            "required": [\n                "location",\n                "Category"\n            ]\n

In [51]:
response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
response['message']['content']

"I found a hotel for you. It's called Parkside Inn and it's located at 424 Por La Mar, Santa Barbara 93103"